# InceptionV3 model initial training

In [2]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import os
import numpy as np
from glob import glob
import json

from keras.applications.inception_v3 import InceptionV3

In [3]:
def extract_InceptionV3(tensor):
    from keras.applications.inception_v3 import InceptionV3, preprocess_input
    return Inceptionv3_model_base.predict(preprocess_input(tensor))

def InceptionV3_predict_labels(img_path):
    # extract bottleneck features
    bottleneck_feature = extract_InceptionV3(path_to_tensor(img_path))
    # return label that is predicted by the model
    return np.argmax(Xception_model.predict(bottleneck_feature))

def InceptionV3_feature_vec(img_path):
    # extract bottleneck features
    return extract_InceptionV3(path_to_tensor(img_path))



In [4]:
train_targets = np.load(open('./images/train_targets.npy','rb'))
valid_targets = np.load(open('./images/valid_targets.npy','rb'))
test_targets = np.load(open('./images/test_targets.npy','rb'))

original_train = np.load(open('./images/original_train.npy','rb'))
original_validation = np.load(open('./images/original_validation.npy','rb'))
original_test = np.load(open('./images/original_test.npy', 'rb'))

bottleneck_Inception_train = np.load(open('./bottleneck_features/bottleneck_Inception_train.npy','rb'))
bottleneck_Inception_validation = np.load(open('./bottleneck_features/bottleneck_Inception_validation.npy','rb'))
bottleneck_Inception_test = np.load(open('./bottleneck_features/bottleneck_Inception_test.npy','rb'))

In [5]:
from keras import backend as K

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

def assessAccuracy(model, features, targets):
    predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in features]
    test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(targets, axis=1))/len(predictions)
    print('Test accuracy: %.4f%%' % test_accuracy)
    
    from sklearn.metrics import f1_score
    print('f1_score:')
    print(f1_score([np.argmax(r) for r in targets], predictions))
    
    from sklearn.metrics import confusion_matrix
    print('confusion matrix')
    print(confusion_matrix([np.argmax(r) for r in targets], predictions))
    
    from sklearn.metrics import average_precision_score
    print('Precision')
    print(average_precision_score([np.argmax(r) for r in targets], predictions))
    
    from sklearn.metrics import recall_score
    print('Recall')
    print(recall_score([np.argmax(r) for r in targets], predictions))

## Initial training of InceptionV3 model using bottleneck features

In [6]:

# define Xception model
inception_model_base = InceptionV3(weights='imagenet', include_top=False)

In [7]:
# InceptionV3 1
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential


inception_model = Sequential()
inception_model.add(Flatten(input_shape=bottleneck_Inception_train.shape[1:]))
inception_model.add(Dropout(0.5))
inception_model.add(Dense(2, activation='softmax'))

inception_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 51200)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 102402    
Total params: 102,402.0
Trainable params: 102,402.0
Non-trainable params: 0.0
_________________________________________________________________


In [50]:
### TODO: Compile the model.
inception_model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=[f1])

In [51]:
from keras.callbacks import ModelCheckpoint  

### TODO: Train the model.
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.inception1.hdf5', 
                               verbose=1, save_best_only=True)

#weight the importance of the malignant/benign samples so that the model does not become bias to benign which makes up 80% of samples
# 1,2,3 input_class_weight= {0:1., 1:10.}
input_class_weight= {0:1., 1:4.}

inception_model.fit(bottleneck_Inception_train, train_targets, 
          validation_data=(bottleneck_Inception_validation, valid_targets),
          epochs=1000, batch_size=20, class_weight=input_class_weight, callbacks=[checkpointer], verbose=1)

Train on 2000 samples, validate on 150 samples
Epoch 1/1000
2000/2000 [==============================] - 5s - loss: 12.0314 - f1: 0.7920 - val_loss: 3.3129 - val_f1: 0.7933
Epoch 2/1000
2000/2000 [==============================] - 1s - loss: 11.7069 - f1: 0.7550 - val_loss: 3.3229 - val_f1: 0.7867
Epoch 3/1000
2000/2000 [==============================] - 1s - loss: 12.1209 - f1: nan - val_loss: 12.7173 - val_f1: nan
Epoch 4/1000
2000/2000 [==============================] - 1s - loss: 12.9594 - f1: nan - val_loss: 12.7173 - val_f1: nan
Epoch 5/1000
2000/2000 [==============================] - 1s - loss: 11.2766 - f1: nan - val_loss: 11.7949 - val_f1: 0.2467
Epoch 6/1000
2000/2000 [==============================] - 1s - loss: 8.9101 - f1: 0.5735 - val_loss: 5.1318 - val_f1: 0.6533
Epoch 7/1000
2000/2000 [==============================] - 1s - loss: 8.9472 - f1: 0.5435 - val_loss: 6.9685 - val_f1: 0.5533
Epoch 8/1000
2000/2000 [==============================] - 1s - loss: 8.9325 - f1: 0.6

2000/2000 [==============================] - 1s - loss: 3.8075 - f1: 0.8485 - val_loss: 3.3152 - val_f1: 0.7600
Epoch 67/1000
2000/2000 [==============================] - 1s - loss: 3.0448 - f1: 0.8950 - val_loss: 2.8833 - val_f1: 0.8067
Epoch 68/1000
2000/2000 [==============================] - 1s - loss: 3.4573 - f1: 0.8685 - val_loss: 3.7042 - val_f1: 0.7600
Epoch 69/1000
2000/2000 [==============================] - 1s - loss: 3.3137 - f1: 0.8735 - val_loss: 2.7567 - val_f1: 0.8067
Epoch 70/1000
2000/2000 [==============================] - 1s - loss: 3.4249 - f1: 0.8675 - val_loss: 3.6994 - val_f1: 0.7600
Epoch 71/1000
2000/2000 [==============================] - 1s - loss: 3.3583 - f1: 0.8670 - val_loss: 4.3608 - val_f1: 0.7200
Epoch 72/1000
2000/2000 [==============================] - 1s - loss: 3.5611 - f1: 0.8775 - val_loss: 3.4240 - val_f1: 0.7733
Epoch 73/1000
2000/2000 [==============================] - 1s - loss: 3.2270 - f1: 0.8885 - val_loss: 3.0223 - val_f1: 0.7867
Epoch 

2000/2000 [==============================] - 1s - loss: 2.6390 - f1: 0.9185 - val_loss: 2.7698 - val_f1: 0.8133
Epoch 133/1000
2000/2000 [==============================] - 1s - loss: 2.5716 - f1: 0.9140 - val_loss: 3.0002 - val_f1: 0.8000
Epoch 134/1000
2000/2000 [==============================] - 1s - loss: 2.4771 - f1: 0.9085 - val_loss: 3.1462 - val_f1: 0.7933
Epoch 135/1000
2000/2000 [==============================] - 1s - loss: 2.2986 - f1: 0.9200 - val_loss: 2.8282 - val_f1: 0.8200
Epoch 136/1000
2000/2000 [==============================] - 1s - loss: 2.2963 - f1: 0.9225 - val_loss: 3.0008 - val_f1: 0.8000
Epoch 137/1000
2000/2000 [==============================] - 1s - loss: 2.0097 - f1: 0.9340 - val_loss: 2.6353 - val_f1: 0.8200
Epoch 138/1000
2000/2000 [==============================] - 1s - loss: 2.2796 - f1: 0.9255 - val_loss: 2.9457 - val_f1: 0.8000
Epoch 139/1000
2000/2000 [==============================] - 1s - loss: 2.3503 - f1: 0.9085 - val_loss: 3.3367 - val_f1: 0.7733

2000/2000 [==============================] - 1s - loss: 2.0139 - f1: 0.9275 - val_loss: 3.3258 - val_f1: 0.7600
Epoch 199/1000
2000/2000 [==============================] - 1s - loss: 2.0166 - f1: 0.9310 - val_loss: 3.0098 - val_f1: 0.8067
Epoch 200/1000
2000/2000 [==============================] - 1s - loss: 1.9192 - f1: 0.9365 - val_loss: 2.8802 - val_f1: 0.8200
Epoch 201/1000
2000/2000 [==============================] - 1s - loss: 2.1960 - f1: 0.9355 - val_loss: 2.8307 - val_f1: 0.8200
Epoch 202/1000
2000/2000 [==============================] - 1s - loss: 1.8903 - f1: 0.9410 - val_loss: 3.0350 - val_f1: 0.8067
Epoch 203/1000
2000/2000 [==============================] - 1s - loss: 1.7652 - f1: 0.9410 - val_loss: 3.7225 - val_f1: 0.7600
Epoch 204/1000
2000/2000 [==============================] - 1s - loss: 1.9156 - f1: 0.9315 - val_loss: 2.8941 - val_f1: 0.8133
Epoch 205/1000
2000/2000 [==============================] - 1s - loss: 1.7574 - f1: 0.9445 - val_loss: 3.5880 - val_f1: 0.7667

2000/2000 [==============================] - 1s - loss: 1.4994 - f1: 0.9595 - val_loss: 3.0700 - val_f1: 0.8000
Epoch 265/1000
2000/2000 [==============================] - 1s - loss: 1.6727 - f1: 0.9575 - val_loss: 2.7623 - val_f1: 0.8200
Epoch 266/1000
2000/2000 [==============================] - 1s - loss: 1.6739 - f1: 0.9495 - val_loss: 3.2421 - val_f1: 0.7667
Epoch 267/1000
2000/2000 [==============================] - 1s - loss: 1.8322 - f1: 0.9335 - val_loss: 3.3497 - val_f1: 0.7867
Epoch 268/1000
2000/2000 [==============================] - 1s - loss: 1.4334 - f1: 0.9570 - val_loss: 3.2890 - val_f1: 0.7800
Epoch 269/1000
2000/2000 [==============================] - 1s - loss: 1.6548 - f1: 0.9560 - val_loss: 3.2489 - val_f1: 0.7933
Epoch 270/1000
2000/2000 [==============================] - 1s - loss: 1.5763 - f1: 0.9500 - val_loss: 3.4518 - val_f1: 0.7800
Epoch 271/1000
2000/2000 [==============================] - 1s - loss: 1.6760 - f1: 0.9480 - val_loss: 3.6898 - val_f1: 0.7400

...

2000/2000 [==============================] - 1s - loss: 1.1205 - f1: 0.9690 - val_loss: 3.5395 - val_f1: 0.7733
Epoch 463/1000
2000/2000 [==============================] - 1s - loss: 1.1912 - f1: 0.9650 - val_loss: 3.0276 - val_f1: 0.8000
Epoch 464/1000
2000/2000 [==============================] - 1s - loss: 1.3035 - f1: 0.9720 - val_loss: 3.5192 - val_f1: 0.7667
Epoch 465/1000
2000/2000 [==============================] - 1s - loss: 1.1497 - f1: 0.9670 - val_loss: 3.3935 - val_f1: 0.7733
Epoch 466/1000
2000/2000 [==============================] - 1s - loss: 1.2161 - f1: 0.9655 - val_loss: 3.1954 - val_f1: 0.8000
Epoch 467/1000
2000/2000 [==============================] - 1s - loss: 1.1985 - f1: 0.9745 - val_loss: 3.3602 - val_f1: 0.7733
Epoch 468/1000
2000/2000 [==============================] - 1s - loss: 1.1676 - f1: 0.9650 - val_loss: 3.4308 - val_f1: 0.7800
Epoch 469/1000
2000/2000 [==============================] - 1s - loss: 0.9884 - f1: 0.9725 - val_loss: 2.8580 - val_f1: 0.8133

2000/2000 [==============================] - 1s - loss: 1.0598 - f1: 0.9725 - val_loss: 3.2834 - val_f1: 0.7933
Epoch 529/1000
2000/2000 [==============================] - 1s - loss: 1.0188 - f1: 0.9705 - val_loss: 3.2124 - val_f1: 0.7933
Epoch 530/1000
2000/2000 [==============================] - 1s - loss: 1.1961 - f1: 0.9705 - val_loss: 3.2615 - val_f1: 0.7933
Epoch 531/1000
2000/2000 [==============================] - 1s - loss: 1.0907 - f1: 0.9720 - val_loss: 3.1459 - val_f1: 0.7867
Epoch 532/1000
2000/2000 [==============================] - 1s - loss: 1.1602 - f1: 0.9635 - val_loss: 3.1270 - val_f1: 0.8000
Epoch 533/1000
2000/2000 [==============================] - 1s - loss: 1.1008 - f1: 0.9710 - val_loss: 3.3185 - val_f1: 0.7867
Epoch 534/1000
2000/2000 [==============================] - 1s - loss: 1.0852 - f1: 0.9680 - val_loss: 3.0970 - val_f1: 0.7933
Epoch 535/1000
2000/2000 [==============================] - 1s - loss: 1.0438 - f1: 0.9690 - val_loss: 3.0051 - val_f1: 0.8067

2000/2000 [==============================] - 1s - loss: 1.0598 - f1: 0.9710 - val_loss: 3.2808 - val_f1: 0.7800
Epoch 595/1000
2000/2000 [==============================] - 1s - loss: 0.9583 - f1: 0.9735 - val_loss: 2.9899 - val_f1: 0.8000
Epoch 596/1000
2000/2000 [==============================] - 1s - loss: 1.0226 - f1: 0.9720 - val_loss: 3.3730 - val_f1: 0.7800
Epoch 597/1000
2000/2000 [==============================] - 1s - loss: 0.9198 - f1: 0.9775 - val_loss: 3.3209 - val_f1: 0.7867
Epoch 598/1000
2000/2000 [==============================] - 1s - loss: 1.0963 - f1: 0.9680 - val_loss: 3.4815 - val_f1: 0.7733
Epoch 599/1000
2000/2000 [==============================] - 1s - loss: 0.9077 - f1: 0.9760 - val_loss: 3.4126 - val_f1: 0.7733
Epoch 600/1000
2000/2000 [==============================] - 1s - loss: 1.0710 - f1: 0.9705 - val_loss: 3.4994 - val_f1: 0.7733
Epoch 601/1000
2000/2000 [==============================] - 1s - loss: 1.0270 - f1: 0.9730 - val_loss: 3.0079 - val_f1: 0.8067

2000/2000 [==============================] - 1s - loss: 1.0406 - f1: 0.9670 - val_loss: 3.4366 - val_f1: 0.7733
Epoch 661/1000
2000/2000 [==============================] - 1s - loss: 0.9729 - f1: 0.9735 - val_loss: 3.3205 - val_f1: 0.7733
Epoch 662/1000
2000/2000 [==============================] - 1s - loss: 0.9102 - f1: 0.9770 - val_loss: 3.0993 - val_f1: 0.8067
Epoch 663/1000
2000/2000 [==============================] - 1s - loss: 0.8361 - f1: 0.9785 - val_loss: 3.3997 - val_f1: 0.7800
Epoch 664/1000
2000/2000 [==============================] - 1s - loss: 1.0724 - f1: 0.9635 - val_loss: 3.3404 - val_f1: 0.7867
Epoch 665/1000
2000/2000 [==============================] - 1s - loss: 0.8525 - f1: 0.9800 - val_loss: 3.3958 - val_f1: 0.7800
Epoch 666/1000
2000/2000 [==============================] - 1s - loss: 0.8787 - f1: 0.9780 - val_loss: 3.3162 - val_f1: 0.7933
Epoch 667/1000
2000/2000 [==============================] - 1s - loss: 0.8959 - f1: 0.9785 - val_loss: 3.0967 - val_f1: 0.7933

KeyboardInterrupt: 

In [8]:
inception_model.load_weights('saved_models/weights.best.inception1.hdf5')


In [9]:
assessAccuracy(inception_model, bottleneck_Inception_test, test_targets)

Test accuracy: 80.5000%
f1_score:
0.387434554974
confusion matrix
[[446  35]
 [ 82  37]]
Precision
0.480739962652
Recall
0.310924369748


### Fine tune InceptionV3 model

In [10]:
inception_model_base.summary()
print(len(inception_model_base.layers))

# unfreeze weights of the 5th layer
#layers_to_freeze = 34 
# unfreeze the weights of the 4th and 5th layers.
#layers_to_freeze = 96


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864                                          
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96                                           
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0                                            
___________________________________________________________________________________________

In [11]:
from keras.models import Model
from keras import optimizers
from keras.layers import Input

# create the base pre-trained model
#input_tensor = Input(shape=original_train[0].shape)
#input_tensor = Input(shape=original_train[0].shape)
inception_model_base = InceptionV3(input_shape=original_train[0].shape, weights='imagenet', include_top=False)
inception_model.load_weights('saved_models/weights.best.inception1.hdf5')


for layer in inception_model_base.layers[:249]:
   layer.trainable = False
for layer in inception_model_base.layers[249:]:
   layer.trainable = True    
    
fine_tuned_inception_model = Model(input=inception_model_base.input, output= inception_model(inception_model_base.output))
    
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
fine_tuned_inception_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=[f1])

//anaconda/envs/capstone-project/lib/python3.5/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`
  '` call to the Keras 2 API: ' + signature)


In [83]:
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.inception.final2.hdf5', 
                               verbose=1, save_best_only=True)

#weight the importance of the malignant/benign samples so that the model does not become bias to benign which makes up 80% of samples
input_class_weight= {0:1., 1:4.}

fine_tuned_inception_model.fit(original_train, train_targets, 
          validation_data=(original_validation, valid_targets),
          epochs=1000, batch_size=20, class_weight=input_class_weight, callbacks=[checkpointer], verbose=1)

Train on 2000 samples, validate on 150 samples
Epoch 1/1000
2000/2000 [==============================] - 491s - loss: 10.0792 - f1: 0.7280 - val_loss: 5.2264 - val_f1: 0.6733
Epoch 2/1000
2000/2000 [==============================] - 344s - loss: 10.2100 - f1: 0.5570 - val_loss: 12.2933 - val_f1: 0.2333
Epoch 3/1000
2000/2000 [==============================] - 339s - loss: 9.6384 - f1: 0.5075 - val_loss: 7.1863 - val_f1: 0.5467
Epoch 4/1000
2000/2000 [==============================] - 339s - loss: 9.9523 - f1: 0.4915 - val_loss: 7.6343 - val_f1: 0.5133
Epoch 5/1000
2000/2000 [==============================] - 340s - loss: 9.9304 - f1: 0.4660 - val_loss: 6.0262 - val_f1: 0.6200
Epoch 6/1000
2000/2000 [==============================] - 337s - loss: 9.6594 - f1: 0.5080 - val_loss: 4.8538 - val_f1: 0.6933
Epoch 7/1000
2000/2000 [==============================] - 338s - loss: 9.3671 - f1: 0.5830 - val_loss: 4.4343 - val_f1: 0.6867
Epoch 8/1000
2000/2000 [==============================] - 337

2000/2000 [==============================] - 338s - loss: 2.5581 - f1: 0.9360 - val_loss: 2.4871 - val_f1: 0.8133
Epoch 64/1000
2000/2000 [==============================] - 335s - loss: 2.5027 - f1: 0.9380 - val_loss: 2.9854 - val_f1: 0.7667
Epoch 65/1000
2000/2000 [==============================] - 336s - loss: 2.3529 - f1: 0.9395 - val_loss: 2.2924 - val_f1: 0.8333
Epoch 66/1000
2000/2000 [==============================] - 336s - loss: 2.5291 - f1: 0.9350 - val_loss: 2.6638 - val_f1: 0.7933
Epoch 67/1000
2000/2000 [==============================] - 336s - loss: 2.4512 - f1: 0.9415 - val_loss: 2.7503 - val_f1: 0.7867
Epoch 68/1000
2000/2000 [==============================] - 335s - loss: 2.1898 - f1: 0.9440 - val_loss: 2.4897 - val_f1: 0.8133
Epoch 69/1000
2000/2000 [==============================] - 336s - loss: 2.5027 - f1: 0.9345 - val_loss: 2.7888 - val_f1: 0.7933
Epoch 70/1000
2000/2000 [==============================] - 335s - loss: 2.4391 - f1: 0.9440 - val_loss: 2.8771 - val_f

2000/2000 [==============================] - 334s - loss: 1.5220 - f1: 0.9620 - val_loss: 2.3911 - val_f1: 0.8067
Epoch 97/1000
2000/2000 [==============================] - 332s - loss: 1.4657 - f1: 0.9595 - val_loss: 2.6675 - val_f1: 0.8067
Epoch 98/1000
2000/2000 [==============================] - 333s - loss: 1.5586 - f1: 0.9605 - val_loss: 2.6106 - val_f1: 0.8067
Epoch 99/1000
2000/2000 [==============================] - 332s - loss: 1.8863 - f1: 0.9225 - val_loss: 2.7886 - val_f1: 0.7800
Epoch 100/1000
2000/2000 [==============================] - 333s - loss: 1.5495 - f1: 0.9455 - val_loss: 2.6374 - val_f1: 0.8067
Epoch 101/1000
2000/2000 [==============================] - 333s - loss: 1.3717 - f1: 0.9700 - val_loss: 2.5586 - val_f1: 0.8000
Epoch 102/1000
2000/2000 [==============================] - 333s - loss: 1.3665 - f1: 0.9715 - val_loss: 2.4510 - val_f1: 0.8067
Epoch 103/1000
2000/2000 [==============================] - 333s - loss: 1.3223 - f1: 0.9735 - val_loss: 2.3737 - v

2000/2000 [==============================] - 333s - loss: 0.9599 - f1: 0.9820 - val_loss: 3.0163 - val_f1: 0.7733
Epoch 163/1000
2000/2000 [==============================] - 333s - loss: 0.8656 - f1: 0.9840 - val_loss: 2.7053 - val_f1: 0.7933
Epoch 164/1000
2000/2000 [==============================] - 333s - loss: 0.8513 - f1: 0.9855 - val_loss: 2.7128 - val_f1: 0.8067
Epoch 165/1000
2000/2000 [==============================] - 333s - loss: 0.8159 - f1: 0.9860 - val_loss: 2.5973 - val_f1: 0.8133
Epoch 166/1000
2000/2000 [==============================] - 333s - loss: 0.8573 - f1: 0.9850 - val_loss: 2.5496 - val_f1: 0.8200
Epoch 167/1000
2000/2000 [==============================] - 333s - loss: 0.8282 - f1: 0.9860 - val_loss: 2.3670 - val_f1: 0.8200
Epoch 168/1000
2000/2000 [==============================] - 333s - loss: 0.8213 - f1: 0.9865 - val_loss: 2.6492 - val_f1: 0.7933
Epoch 169/1000
2000/2000 [==============================] - 333s - loss: 0.8456 - f1: 0.9830 - val_loss: 2.6077 

2000/2000 [==============================] - 333s - loss: 1.3741 - f1: 0.9295 - val_loss: 2.5631 - val_f1: 0.8267
Epoch 228/1000
2000/2000 [==============================] - 333s - loss: 0.6557 - f1: 0.9805 - val_loss: 2.5825 - val_f1: 0.8267
Epoch 229/1000
2000/2000 [==============================] - 333s - loss: 0.5987 - f1: 0.9860 - val_loss: 2.4709 - val_f1: 0.8267
Epoch 230/1000
2000/2000 [==============================] - 333s - loss: 0.6385 - f1: 0.9875 - val_loss: 2.4185 - val_f1: 0.8400
Epoch 231/1000
2000/2000 [==============================] - 333s - loss: 0.6623 - f1: 0.9875 - val_loss: 2.4839 - val_f1: 0.8200
Epoch 232/1000
2000/2000 [==============================] - 333s - loss: 0.6633 - f1: 0.9825 - val_loss: 2.5616 - val_f1: 0.8000
Epoch 233/1000
2000/2000 [==============================] - 333s - loss: 0.5538 - f1: 0.9835 - val_loss: 2.5973 - val_f1: 0.8000
Epoch 234/1000
2000/2000 [==============================] - 333s - loss: 0.6189 - f1: 0.9750 - val_loss: 2.6028 

KeyboardInterrupt: 

In [12]:
#fine_tuned_inception_model.load_weights('saved_models/weights.best.inception.final.hdf5')
fine_tuned_inception_model.load_weights('saved_models/weights.best.inception.final2.hdf5')

In [13]:
assessAccuracy(fine_tuned_inception_model, original_test, test_targets)

Test accuracy: 79.5000%
f1_score:
0.448430493274
confusion matrix
[[427  54]
 [ 69  50]]
Precision
0.507968648998
Recall
0.420168067227


In [14]:

fine_tuned_inception_model.load_weights('saved_models/weights.best.inception.final.hdf5')
assessAccuracy(fine_tuned_inception_model, original_test, test_targets)

Test accuracy: 79.5000%
f1_score:
0.46288209607
confusion matrix
[[424  57]
 [ 66  53]]
Precision
0.518598166539
Recall
0.445378151261



### Generate predictions and store for training/validation
to be used for training our random forrest classifier, and test to be used to validate against.

In [87]:
fine_tuned_inception_model.load_weights('saved_models/weights.best.inception.final.hdf5')


In [110]:
train_pred = fine_tuned_inception_model.predict(original_train)

In [100]:
val_pred = fine_tuned_inception_model.predict(original_validation)

In [101]:
test_pred = fine_tuned_inception_model.predict(original_test)

In [127]:
np.save(open('./cnn_output/inception_train.npy', 'wb'), train_pred)
np.save(open('./cnn_output/inception_validation.npy', 'wb'), val_pred)
np.save(open('./cnn_output/inception_test.npy', 'wb'), test_pred)

In [133]:
train_pred_pre = inception_model.predict(bottleneck_Inception_train)
val_pred_pre = inception_model.predict(bottleneck_Inception_validation)
test_pred_pre = inception_model.predict(bottleneck_Inception_test)

In [134]:
np.save(open('./cnn_output/inception_pre_train.npy', 'wb'), train_pred_pre)
np.save(open('./cnn_output/inception_pre_validation.npy', 'wb'), val_pred_pre)
np.save(open('./cnn_output/inception_pre_test.npy', 'wb'), test_pred_pre)